In [1]:
!pip install gurobipy pandas openpyxl

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 1.6 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/11.1 MB 1.8 MB/s eta 0:00:06
   ----- ---------------------------------- 1.6/11.1 MB 2.1 MB/s eta 0:00:05
   ------ --------------------------------- 1.8/11.1 MB 2.2 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.1 MB 2.0 MB/s eta 0:00:05
   -------- ------------------------------- 2.4/11.1 MB 1.8 MB/s eta 0:00:05
   ---------- ----------------------------- 2.9/11.1 MB 1.8 MB/s eta 0:00:05
   ----------- ---------------------------- 3.1/11.1 MB 1.9 MB/s eta 0:00:05
   ------------- -------------------------- 3.7/11.1 MB 1.8 MB/s eta 0:00:05
   ------------- -------------------------- 3.7/11.1 MB 1.8 MB/s eta 0:00:05
   --------------- -

In [2]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

In [ ]:
# ==== PREPARACIÓN DE DICCIONARIOS PARA CONSTRUIR CONJUNTOS Y PARÁMETROS ===

# === 1. LEER DATOS DE ENTRADA ===
archivo = "INPUT MODELO OPTIMIZACIÓN.xlsx"
df_granjas = pd.read_excel(archivo, sheet_name="Input_Granjas")
df_demanda = pd.read_excel(archivo, sheet_name="Input_Demanda", header=4)
df_raw = pd.read_excel(archivo, sheet_name="Input_Demanda", header=None)

# === PARÁMETROS ===
W, ADG, CAT, N, TIPO = {}, {}, {}, {}, {}

for _, row in df_granjas.iterrows():
    g, p, s = row["GRANJA"], row["GALPON"], row["SEXO"]
    N[(g, p, s)] = row["SALDO"]
    TIPO[(g, p, s)] = row["TIPO"]

    for d in range(1, 7):
        col_w = 'WEIGTH' if d == 1 else f'WEIGTH{d}'
        col_adg = 'ADG' if d == 1 else f'ADG{d}'
        col_cat = 'CAT PESO' if d == 1 else f'CAT PESO{d}'

        if pd.notnull(row[col_w]): W[(g, p, s, d)] = float(row[col_w])
        if pd.notnull(row[col_adg]): ADG[(g, p, s, d)] = float(row[col_adg])
        if pd.notnull(row[col_cat]): CAT[(g, p, s, d)] = row[col_cat].strip().upper()

# === DEMANDA Q[tipo, cat, d] ===
Q = {}
tipos_validos = ["CAMPESINO", "BLANCO"]
cats_validas = ["PEQUEÑO", "MEDIANO", "GRANDE"]

for _, row in df_demanda.iterrows():
    t, c = row["Tipo"], row["Categoria Peso"]
    if t in tipos_validos and c in cats_validas:
        for d in range(1, 22):
            val = row.get(f"Day{d}")
            if pd.notnull(val):
                Q[(t, c, d)] = val

# === MAPD y TOT ===
MAPD = {}
for i in range(13, 34):
    semana = df_raw.iloc[i, 1]
    dia = df_raw.iloc[i, 2]
    if pd.notnull(semana) and pd.notnull(dia):
        MAPD[int(dia)] = int(semana)

TOT = {}
val_tot = df_raw.iloc[15, 5]
if pd.notnull(val_tot): TOT[1] = float(val_tot)

tol = float(df_raw.iloc[1, 2])  # Tolerancia


In [9]:
# === 2. DEFINICIÓN DE CONJUNTOS ===


# Conjunto de grupos: (granja, galpón, sexo)
GP = list(N.keys())

# Conjunto de días en el horizonte (por ejemplo 1 a 6)
D = sorted(set(d for (_, _, _, d) in W.keys()))

# Conjunto de tipos de pollo
T = sorted(set(TIPO.values()))

# Conjunto de categorías de peso
C = sorted(set(c for (_, _, _, _), c in CAT.items()))

# Conjunto de semanas (extraído del mapeo día → semana)
WEEK = sorted(set(MAPD[d] for d in MAPD))


In [10]:
# === 3. CREACION DEL MODELO ===

# === MODELO VACÍO ===
model = Model("modelo_cosecha_pollos")

# === VARIABLES ===

# x[g, p, s, d] = 1 si el grupo se cosecha el día d
x = model.addVars(GP, D, vtype=GRB.BINARY, name="x")

# dev[t, c, d] = desviación absoluta entre cosecha y demanda para ese tipo, categoría y día
dev = model.addVars(T, C, D, vtype=GRB.CONTINUOUS, lb=0, name="dev")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2680166
Academic license 2680166 - for non-commercial use only - registered to 10___@u.icesi.edu.co


In [11]:
# === 4. FUNCIÓN OBJETIVO ===

model.setObjective(
    quicksum(dev[t, c, d] for t in T for c in C for d in D),
    GRB.MINIMIZE
)


In [12]:
# === 5. PLANTEAMIENTO DE LAS RESTRICCIONES ===

# === R1: Cada grupo se cosecha 1 sola vez

for g, p, s in GP:
    model.addConstr(
        quicksum(x[g, p, s, d] for d in D) <= 1,
        name=f"R1_una_vez_{g}_{p}_{s}"
    )

# === R2: la diferencia absoluta entre lo cosechado y la demanda debe ser menor a la desviacion ===

for t in T:
    for c in C:
        for d in D:
            cantidad_real = quicksum(
                N[gps] * x[gps + (d,)] # Asegura que el grupo haya sido cosechado, y agarra la cantidad del grupo (N)
                for gps in GP
                if TIPO[gps] == t and CAT.get(gps + (d,), None) == c #Asegura que las categorias y tipos coincidan
            )
            if (t, c, d) in Q:
                model.addConstr(cantidad_real - Q[t, c, d] <= dev[t, c, d], # Modelado de la restricción en absoluto
                                name=f"R2_sup_{t}_{c}_{d}")
                model.addConstr(Q[t, c, d] - cantidad_real <= dev[t, c, d], # Modelado de la restricción en absoluto
                                name=f"R2_inf_{t}_{c}_{d}")

# === R3: Para los grupos mixtos, se prioriza cosechar las hembras sobre los machos ===

galpones = set((g, p) for g, p, s in GP)

for g, p in galpones:
    if ((g, p, 'HEMBRA') in GP) and ((g, p, 'MACHO') in GP):
        for d in D:
            if d >= 5: #Solo tiene sentido si han transcurrido más de 4 días 
                suma_hembras_previos = quicksum(
                    x[g, p, 'HEMBRA', dp]
                    for dp in range(d - 4, d) if dp in D #Valida si las hembras de un galpon mixto se cosechan durante 4 días anteriores
                )
                model.addConstr(
                    x[g, p, 'MACHO', d] <= 1 - suma_hembras_previos,
                    name=f"R3_prioridad_{g}_{p}_{d}" #Evita que se cosechen los machos si no han pasado 4 días o más desde la cosecha de hembras
                )

# === R4: Asegura que la cosecha de la semana esté dentro del rango de tolerancia de la demanda total para esa semana ===

for w in WEEK:
    dias_semana = [d for d in D if MAPD.get(d) == w]
    
    suma_cosechada = quicksum(
        N[gps] * x[gps + (d,)] #Computa cuantos pollos sales por grupo cosechado
        for gps in GP
        for d in dias_semana
    )
    
    if w in TOT:
        model.addConstr(
            suma_cosechada >= (1 - tol) * TOT[w], #Límite inferior de la demanda total semanal
            name=f"R5_inferior_{w}"
        )
        model.addConstr(
            suma_cosechada <= (1 + tol) * TOT[w], #Límite superior de la demanda total semanal
            name=f"R5_superior_{w}"
        )


In [13]:
# Resolver el modelo
model.optimize()


Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2680166 - for non-commercial use only - registered to 10___@u.icesi.edu.co
Optimize a model with 222 rows, 924 columns and 4512 nonzeros
Model fingerprint: 0x845d5440
Variable types: 36 continuous, 888 integer (888 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 961788.00000
Presolve removed 47 rows and 136 columns
Presolve time: 0.01s
Presolved: 175 rows, 788 columns, 3790 nonzeros
Found heuristic solution: objective 765328.00000
Variable types: 0 continuous, 788 integer (758 binary)
Found heuristic solution: objective 603919.00000

Root relaxation: objective 4.14768

In [14]:
#Revisar el estado del modelo de optimizacion

if model.status == GRB.OPTIMAL:
    print("\n✅ Solución óptima encontrada.")
elif model.status == GRB.INFEASIBLE:
    print("\n❌ El modelo es infactible.")
    model.computeIIS()
    model.write("modelo_infactible.ilp")
elif model.status == GRB.UNBOUNDED:
    print("\n❌ El modelo no está acotado.")
else:
    print(f"\n⚠️ El modelo terminó con estado: {model.status}")



✅ Solución óptima encontrada.


In [15]:
#Resultados de la cosecha

print("\n📦 Grupos cosechados:")
for g, p, s in GP:
    for d in D:
        var = x[g, p, s, d]
        if var.X > 0.5:
            print(f"Grupo ({g}, {p}, {s}) cosechado el día {d}")



📦 Grupos cosechados:
Grupo (MI GRANJITA, 4, M) cosechado el día 5
Grupo (EL DESCANSO, 8, M) cosechado el día 5
Grupo (MONTERREY, 5, M) cosechado el día 4
Grupo (GRANADILLO, 2, M) cosechado el día 5
Grupo (GRANADILLO, 3, M) cosechado el día 2
Grupo (GRANADILLO, 4, M) cosechado el día 1
Grupo (GRANADILLO, 8, M) cosechado el día 6
Grupo (EL OASIS, 1, M) cosechado el día 5
Grupo (EL OASIS, 1, H) cosechado el día 6
Grupo (EL OASIS, 2, H) cosechado el día 4
Grupo (EL OASIS, 3, M) cosechado el día 6
Grupo (EL OASIS, 3, H) cosechado el día 1
Grupo (EL OASIS, 4, H) cosechado el día 6
Grupo (EL OASIS, 5, H) cosechado el día 1
Grupo (JR, 1, M) cosechado el día 1
Grupo (JR, 1, H) cosechado el día 4
Grupo (JR, 2, M) cosechado el día 4
Grupo (JR, 2, H) cosechado el día 3
Grupo (JR, 3, M) cosechado el día 3
Grupo (JR, 3, H) cosechado el día 4
Grupo (JR, 4, M) cosechado el día 2
Grupo (JR, 4, H) cosechado el día 5
Grupo (JR, 5, M) cosechado el día 2
Grupo (JR, 5, H) cosechado el día 1
Grupo (GIRASOL,

In [16]:
#Desviaciones presentadas

print("\n📉 Desviaciones en el cumplimiento del mix diario:")
for t in T:
    for c in C:
        for d in D:
            val = dev[t, c, d].X
            if val > 0.01:
                print(f"Tipo {t}, Categoría {c}, Día {d}: desviación = {val:.0f} aves")



📉 Desviaciones en el cumplimiento del mix diario:
Tipo BLANCO, Categoría GRANDE, Día 1: desviación = 15000 aves
Tipo BLANCO, Categoría GRANDE, Día 2: desviación = 1240 aves
Tipo BLANCO, Categoría GRANDE, Día 3: desviación = 15000 aves
Tipo BLANCO, Categoría GRANDE, Día 4: desviación = 6270 aves
Tipo BLANCO, Categoría GRANDE, Día 5: desviación = 2630 aves
Tipo BLANCO, Categoría GRANDE, Día 6: desviación = 1580 aves
Tipo BLANCO, Categoría MEDIANO, Día 1: desviación = 31460 aves
Tipo BLANCO, Categoría MEDIANO, Día 2: desviación = 67815 aves
Tipo BLANCO, Categoría MEDIANO, Día 3: desviación = 49403 aves
Tipo BLANCO, Categoría MEDIANO, Día 4: desviación = 20670 aves
Tipo BLANCO, Categoría MEDIANO, Día 5: desviación = 35629 aves
Tipo BLANCO, Categoría MEDIANO, Día 6: desviación = 10395 aves
Tipo BLANCO, Categoría PEQUEÑO, Día 1: desviación = 8371 aves
Tipo BLANCO, Categoría PEQUEÑO, Día 2: desviación = 99 aves
Tipo BLANCO, Categoría PEQUEÑO, Día 3: desviación = 757 aves
Tipo BLANCO, Categor

In [17]:
#Guardar resultados en un CSV
import csv

with open("resultados_cosecha.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Granja", "Galpón", "Sexo", "Día", "Cosechado (1/0)"])
    for g, p, s in GP:
        for d in D:
            writer.writerow([g, p, s, d, int(x[g, p, s, d].X > 0.5)])
